In [20]:
# import statements
from time import sleep
from json import dumps
from kafka3 import KafkaProducer
import random
from datetime import date, datetime, timedelta
from pprint import pprint
import pandas as pd
import pymongo
from pymongo import MongoClient
import pandas as pd

hostip = "192.168.0.130" # change it to your IP


def publish_message(producer_instance, topic_name, data):
    try:
        value =  bytes(str(data), 'utf-8')
        producer_instance.send(topic_name, value)
        print('Message published successfully. Data: ' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=[f'{hostip}:9092'],
                                  #value_serializer=lambda x: dumps(x, default=str).encode('utf-8'),
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer
    
if __name__ == '__main__':
          
    client = MongoClient('192.168.0.130', 27017) 
    db = client.fit3182_db
    fit3182_assignment_db = db.fit3182_assignment_db
    
    res = fit3182_assignment_db.aggregate([
    { "$sort" : {"date" : -1 }},
    { "$limit" : 1 }
    ])
    
    
    
    # load data
    climate_streaming_file = pd.read_csv("./climate_streaming.csv")
    #climate_file = climate_file.apply(pd.to_numeric, errors='ignore')
    climate_data = climate_streaming_file.to_dict(orient="records")

    #pprint(climate_data)
    
    topic = 'Main_Stream'
    
    print('Publishing records..')
    producer = connect_kafka_producer()
    mydate = list(res)[0]['date'] + timedelta(1)  # lastest climate date in mongoDB is 2024-01-01 so newdate should be 2024-01-02
    
    for e in range(100):
        data = climate_data[random.randrange(0, len(climate_data))]
        message = mydate.strftime("%x") + ", " + str("producer1") + " , " + str(data['latitude']) + " , "  + str(data['longitude']) + " , " + str(data['air_temperature_celcius']) + " / " + str(data['relative_humidity']) + " / " + str(data['windspeed_knots'])  + " / " + str(data['max_wind_speed']) + " / " + str(data['precipitation ']) + " / " + str(data['GHI_w/m2']) 
        publish_message(producer, topic, message) # " "parsed" is the key, data is the value"
        sleep(10)
        mydate = timedelta(1) + mydate
    

Publishing records..
Message published successfully. Data: 01/02/24, producer1 , -36.834 , 142.524 , 13 / 50.3 / 8.6 / 13.0 /  0.02G / 111
Message published successfully. Data: 01/03/24, producer1 , -37.34 , 143.123 , 19 / 52.8 / 12.5 / 21.0 /  0.47G / 159
Message published successfully. Data: 01/04/24, producer1 , -37.59 , 149.31 , 19 / 57.2 / 8.9 / 15.0 /  0.98G / 153
Message published successfully. Data: 01/05/24, producer1 , -36.748 , 144.168 , 11 / 48.2 / 10.8 / 22.9 /  0.59G / 96
Message published successfully. Data: 01/06/24, producer1 , -37.719 , 142.154 , 13 / 47.0 / 7.9 / 15.0 /  0.02G / 114
Message published successfully. Data: 01/07/24, producer1 , -36.942 , 143.282 , 25 / 58.3 / 7.1 / 13.0 /  0.00I / 199
Message published successfully. Data: 01/08/24, producer1 , -36.8264 , 142.6138 , 15 / 48.2 / 14.7 / 25.1 /  0.43G / 131
Message published successfully. Data: 01/09/24, producer1 , -37.618 , 143.0013 , 14 / 36.0 / 15.9 / 22.0 /  0.00I / 134
Message published successfully. 

KeyboardInterrupt: 